In [1]:
!pip install transformers datasets torch streamlit gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
import gradio as gr

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
from datasets import Dataset
import torch
import random
import json
import gradio as gr


In [8]:
import json

# Define dataset with 20 incorrect math memes

memes = [
    {"wrong": "8 ÷ 2(2+2) = 1", "correct": "8 ÷ 2(2+2) = 16", "explanation": "Using PEMDAS: 8 ÷ 2(4) → 8 ÷ 2 × 4 → 4 × 4 = 16."},
    {"wrong": "5² = 10", "correct": "5² = 25", "explanation": "5² means 5 × 5, not 5 × 2."},
    {"wrong": "6/2(1+2) = 1", "correct": "6/2(1+2) = 9", "explanation": "Solve parentheses first: 6/2(3) → 6 ÷ 2 × 3 → 3 × 3 = 9."},
    {"wrong": "√25 = ±5", "correct": "√25 = 5", "explanation": "The principal square root of 25 is 5. '±' applies when solving x² = 25."},
    {"wrong": "2+3×4 = 20", "correct": "2+3×4 = 14", "explanation": "Multiplication comes before addition: 3×4 = 12, then 2+12 = 14."},
    {"wrong": "10 ÷ 5(2) = 1", "correct": "10 ÷ 5(2) = 4", "explanation": "10 ÷ 5 × 2 → 2 × 2 = 4 (left to right)."},
    {"wrong": "(3 + 3)² = 9", "correct": "(3 + 3)² = 36", "explanation": "(6)² = 36, not 3² + 3²."},
    {"wrong": "12 ÷ 4 + 2 = 1", "correct": "12 ÷ 4 + 2 = 5", "explanation": "12 ÷ 4 = 3, then 3 + 2 = 5."},
    {"wrong": "7 - 2(3) = 5", "correct": "7 - 2(3) = 1", "explanation": "Multiplication first: 2(3) = 6, then 7 - 6 = 1."},
    {"wrong": "15 ÷ 3 × 2 = 2.5", "correct": "15 ÷ 3 × 2 = 10", "explanation": "Divide first: 15 ÷ 3 = 5, then 5 × 2 = 10."},
    {"wrong": "4² + 4² = 8²", "correct": "4² + 4² ≠ 8²", "explanation": "16 + 16 = 32, which is not equal to 8² = 64."},
    {"wrong": "50% of 100 is 75", "correct": "50% of 100 is 50", "explanation": "50% means 100 × 0.5 = 50."},
    {"wrong": "3² + 4² = 5²", "correct": "3² + 4² = 25", "explanation": "3² + 4² = 9 + 16 = 25, not 5²."},
    {"wrong": "1+1 = 3", "correct": "1+1 = 2", "explanation": "Basic math! 1+1 always equals 2."},
    {"wrong": "2(3+4) = 2×3+4", "correct": "2(3+4) = 2×3+2×4 = 14", "explanation": "Distribute 2: 2×3 + 2×4 = 6 + 8 = 14."},
    {"wrong": "9 ÷ 3(1+2) = 1", "correct": "9 ÷ 3(1+2) = 9", "explanation": "9 ÷ 3 × 3 = 3 × 3 = 9."},
    {"wrong": "100 - 50% = 0", "correct": "100 - 50% = 50", "explanation": "50% of 100 is 50, so 100 - 50 = 50."},
    {"wrong": "10 × 0 = 10", "correct": "10 × 0 = 0", "explanation": "Anything multiplied by 0 is always 0."},
    {"wrong": "√(-1) = 1", "correct": "√(-1) = i", "explanation": "√(-1) is the imaginary unit 'i', not 1."}
]



# Save dataset as JSON
with open("meme_dataset.json", "w") as f:
    json.dump(memes, f)

print("Dataset saved successfully with 20 math memes!")


Dataset saved successfully with 20 math memes!


In [9]:
import json

# Load and print dataset
with open("meme_dataset.json", "r") as f:
    memes = json.load(f)

# Print first entry
print(memes[0])  # ✅ This should contain "wrong", "correct", and "explanation"


{'wrong': '8 ÷ 2(2+2) = 1', 'correct': '8 ÷ 2(2+2) = 16', 'explanation': 'Using PEMDAS: 8 ÷ 2(4) → 8 ÷ 2 × 4 → 4 × 4 = 16.'}


In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
from datasets import load_dataset

# Load GPT-2 Model and Tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Fix: Add a padding token
tokenizer.pad_token = tokenizer.eos_token

# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained(model_name)

# Load dataset
dataset = load_dataset("json", data_files="meme_dataset.json")["train"]

# Tokenization function
def tokenize_function(example):
    text = f"Incorrect: {example['wrong']} → Correct: {example['correct']} | Explanation: {example['explanation']}"
    tokens = tokenizer(text, truncation=True, padding="max_length", max_length=100)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

# Tokenize dataset correctly
tokenized_datasets = dataset.map(tokenize_function, batched=False)

# Fine-tuning setup
training_args = TrainingArguments(
    output_dir="meme_trainer",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    save_strategy="epoch",
    evaluation_strategy="no",
    report_to="none"
)

trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_datasets)
trainer.train()

# Save fine-tuned model
model.save_pretrained("fine_tuned_meme_model")
tokenizer.save_pretrained("fine_tuned_meme_model")


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


('fine_tuned_meme_model/tokenizer_config.json',
 'fine_tuned_meme_model/special_tokens_map.json',
 'fine_tuned_meme_model/vocab.json',
 'fine_tuned_meme_model/merges.txt',
 'fine_tuned_meme_model/added_tokens.json')

In [12]:
import torch

def fix_math_meme(meme_text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Detect GPU or CPU
    inputs = tokenizer(meme_text, return_tensors="pt").to(device)  # Move inputs to the same device as the model
    model.to(device)  # Move model to the correct device

    output = model.generate(
        **inputs,
        max_length=100,
        temperature=0.8,
        top_p=0.9,
        repetition_penalty=1.3,
        do_sample=True
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test on 3 incorrect math memes
incorrect_memes = [
    "8 ÷ 2(2+2) = 1",
    "5² = 10",
    "100 - 50% = 0"
]

# Print corrected memes
for i, meme in enumerate(incorrect_memes):
    fixed_meme = fix_math_meme(f"Incorrect: {meme} → Correct:")
    print(f"Fixed Meme {i+1}: {fixed_meme}\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Fixed Meme 1: Incorrect: 8 ÷ 2(2+2) = 1 → Correct: 10 ó² + 5 | Explanation for 12 × 3 ≠3 as √(-1): 16 ε ⟨ 4, then 14 ´ - 6 x 9 (18 − 20), which is 18×4.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Fixed Meme 2: Incorrect: 5² = 10 → Correct: 3 × 2 | Explanation of 4 ÷ 6 + 1 is 14.

Fixed Meme 3: Incorrect: 100 - 50% = 0 → Correct: 25 – 20 | Explanation : This is an integer, not 1.



In [14]:


def gradio_fix_math_meme(meme_text):
    return fix_math_meme(f"Incorrect: {meme_text} → Correct:")

iface = gr.Interface(
    fn=gradio_fix_math_meme,
    inputs=gr.Textbox(label="Enter an Incorrect Math Meme"),
    outputs=gr.Textbox(label="Corrected Meme"),
    title="Math Meme Fixer"
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://10c23364cd4ac2bddf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
